In [1]:
import os
import numpy as np
import pandas as pd 
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow.keras.backend as K
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, SeparableConv2D, MaxPool2D, LeakyReLU, Activation
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
import tensorflow as tf

seed = 232
np.random.seed(seed)


In [2]:
input_path = 'C:/Users/shahe/Desktop/alzheimers/'
#'MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented'
for _set in ['train', 'test']:
    n_mild = len(os.listdir(input_path + _set + '/MildDemented'))
    n_moderate = len(os.listdir(input_path + _set + '/ModerateDemented'))
    n_non = len(os.listdir(input_path + _set + '/NonDemented'))
    n_veryMild = len(os.listdir(input_path + _set + '/VeryMildDemented'))
    print('Set: {}, normal images: {}, pneumonia images: {}'.format(_set,n_mild, n_moderate, n_non, n_veryMild))

Set: train, normal images: 717, pneumonia images: 52
Set: test, normal images: 179, pneumonia images: 12


In [3]:
def process_data(img_dims, batch_size):
    # Data generation objects
    train_datagen = ImageDataGenerator(rescale=1./255, zoom_range=0.3, vertical_flip=True)
    test_val_datagen = ImageDataGenerator(rescale=1./255)
    
    # This is fed to the network in the specified batch sizes and image dimensions
    train_gen = train_datagen.flow_from_directory(
    directory=input_path+'train', 
    target_size=(img_dims, img_dims), 
    batch_size=batch_size, 
    class_mode='categorical', 
    shuffle=True)

    test_gen = test_val_datagen.flow_from_directory(
    directory=input_path+'test', 
    target_size=(img_dims, img_dims), 
    batch_size=batch_size, 
    class_mode='categorical', 
    shuffle=True)
    
    # I will be making predictions off of the test set in one batch size
    # This is useful to be able to get the confusion matrix
    test_data = []
    test_labels = []
    #'MildDemented', 'ModerateDemented', 'NonDemented', 'VeryMildDemented'
    for cond in ['/MildDemented/', '/ModerateDemented/', '/NonDemented/', '/VeryMildDemented/']:
        for img in (os.listdir(input_path + 'test' + cond)):
            img = plt.imread(input_path+'test'+cond+img)
            img = cv2.resize(img, (img_dims, img_dims))
            img = np.dstack([img, img, img])
            img = img.astype('float32') / 255
            if cond=='/MildDemented/':
                label = 0
            elif cond=='/ModerateDemented/':
                label = 1
            elif cond=='/NonDemented/':
                label = 2
            elif cond=='/VeryMildDemented/':
                label = 3
            test_data.append(img)
            test_labels.append(label)
        
    test_data = np.array(test_data)
    test_labels = np.array(test_labels)
    
    return train_gen, test_gen, test_data, test_labels

In [7]:
img_dims = 150
epochs = 10
batch_size = 32

train_gen, test_gen, test_data, test_labels = process_data(img_dims, batch_size)

Found 5121 images belonging to 4 classes.
Found 1279 images belonging to 4 classes.


In [16]:
inputs = Input(shape=(img_dims, img_dims, 3))

# First conv block
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(inputs)
x = Conv2D(filters=16, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Second conv block
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Third conv block
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)

# Fourth conv block
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# Fifth conv block
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = SeparableConv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same')(x)
x = BatchNormalization()(x)
x = MaxPool2D(pool_size=(2, 2))(x)
x = Dropout(rate=0.2)(x)

# FC layer
x = Flatten()(x)
x = Dense(units=512, activation='relu')(x)
x = Dropout(rate=0.7)(x)
x = Dense(units=128, activation='relu')(x)
x = Dropout(rate=0.5)(x)
x = Dense(units=64, activation='relu')(x)
x = Dropout(rate=0.3)(x)

# Output layer
output = Dense(units=4, activation='sigmoid')(x)

# Creating model and compiling
model = Model(inputs=inputs, outputs=output)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Callbacks
checkpoint = ModelCheckpoint(filepath='Alzheimers_Weights.h5', save_best_only=True, save_weights_only=True)
lr_reduce = ReduceLROnPlateau(monitor='val_accuracy', factor=0.3, patience=1, verbose=2, mode='auto')
early_stop = EarlyStopping(monitor='val_loss', min_delta=0.1, patience=1, mode='min')

In [ ]:
hist = model.fit_generator(
           train_gen, steps_per_epoch=train_gen.samples // batch_size, 
           epochs=epochs, validation_data=test_gen, 
           validation_steps=test_gen.samples // batch_size, callbacks=[checkpoint, lr_reduce])

Epoch 1/10
160/160 [==============================] - 223s 1s/step - loss: 0.5310 - accuracy: 0.4555 - val_loss: 0.4759 - val_accuracy: 0.5000
Epoch 2/10
160/160 [==============================] - 241s 2s/step - loss: 0.4254 - accuracy: 0.5398 - val_loss: 0.4961 - val_accuracy: 0.4992

Epoch 00002: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.
Epoch 3/10
160/160 [==============================] - 254s 2s/step - loss: 0.4123 - accuracy: 0.5574 - val_loss: 0.4631 - val_accuracy: 0.4992

Epoch 00003: ReduceLROnPlateau reducing learning rate to 9.000000427477062e-05.
Epoch 4/10
160/160 [==============================] - 270s 2s/step - loss: 0.4029 - accuracy: 0.5674 - val_loss: 0.5478 - val_accuracy: 0.5016
Epoch 5/10
160/160 [==============================] - 291s 2s/step - loss: 0.3995 - accuracy: 0.5654 - val_loss: 0.4163 - val_accuracy: 0.5224
Epoch 6/10
160/160 [==============================] - 257s 2s/step - loss: 0.4062 - accuracy: 0.5682 - val_loss: 0.4875 - v